In [6]:
timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))

ply
format binary_little_endian 1.0
element vertex 54010
property float x
property float y
property float z
end_header


In [ ]:
import numpy as np
import torch
from experiment import discount_cumsum

def visualize_performance(model, trajectories, start_position, goal_position):
        model.eval()

        training_positions = []
        testing_positions = []

        for traj in trajectories:
            training_positions.append(traj["observations"][:, -243:-240])  # Extract position information from the last 3 elements of state
            start_position = torch.zeros(1, 1, 3)
            states = start_position
            print(states.shape)
            actions = torch.ones(1, 1, 3)*0.01 # change in the future
            print(actions)
            rewards = -np.linalg.norm(states[0][0] - goal_position) 
            print(rewards)
            
            rtg = discount_cumsum(rewards, 0.99)
            timesteps = np.arange(0, 1).reshape(1, -1)
            for i in range(100):
                 with torch.no_grad():
                    state_preds,action_preds, reward_preds = model.forward(states, actions, rewards, rtg[:, :-1], timesteps)
                    next_state,next_action,next_reward = state_preds[:,-1,:],action_preds[:,-1,:],reward_preds[:,-1,:]
                    states = torch.cat((states,next_state),dim=1)
                    actions = torch.cat((actions,next_action),dim=1)
                    rewards = torch.cat((rewards,next_reward),dim=1)
                    rtg = discount_cumsum(rewards, 0.99)
                    timesteps = np.arange(0, states.shape[1]).reshape(1, -1)

            # (1,100,3)
            print(actions.shape)
            
            vector_to_goal = np.sum(actions.cpu().numpy(),axis=1).reshape(1, -1)+start_position.cpu().numpy()
            distance_to_goal = np.linalg.norm(vector_to_goal)

            # Calculate testing positions from predicted actions
            testing_positions.append(np.cumsum(actions.cpu().numpy(), axis=1)+start_position.cpu().numpy())

        testing_positions = np.concatenate(testing_positions, axis=0)

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Color code for each training trajectory
        colors = cm.rainbow(np.linspace(0, 1, len(trajectories)))

        # Plot training trajectories
        for traj_positions, color in zip(training_positions, colors):
            ax.scatter(traj_positions[:, 0], traj_positions[:, 1], traj_positions[:, 2], c='red', alpha=0.1, marker='o', s=2)

        # Plot testing trajectories
        ax.scatter(testing_positions[:, 0], testing_positions[:, 1], testing_positions[:, 2], c='blue', alpha=0.1, marker='o', s=6)
        # Plot start and goal positions
        ax.scatter(start_position[0], start_position[1], start_position[2], c='green', marker='s', label='Start', s=100)
        ax.scatter(goal_position[0], goal_position[1], goal_position[2], c='purple', marker='*', label='Goal', s=100)

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.legend()

        plt.show()

In [ ]:
normalized_observation = (observation-np.mean(observation,dim=-1,keepdims=True))/np.std(observation,dim=-1,keepdims=True)
normalized_action = (action-np.mean(action,dim=-1,keepdims=True))/np.std(action,dim=-1,keepdims=True)
normalized_reward = (reward-np.mean(reward,dim=-1,keepdims=True))/np.std(reward,dim=-1,keepdims=True)

In [ ]:
import pickle
import plyfile

import torch
import numpy as np
from plyfile import PlyData

import open3d as o3d
import numpy as np


plydata = PlyData.read('000028_lcam_front_lidar.ply')
print(plydata)
x = plydata['vertex']['x']
y = plydata['vertex']['y']
z = plydata['vertex']['z']

# Load point cloud data from file
pcd = o3d.io.read_point_cloud('000028_lcam_front_lidar.ply')
points = np.asarray(pcd.points)

# Set voxel size
voxel_size = 0.1

# Voxelization
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
voxels = np.asarray(voxel_grid.get_voxels())

# Convert voxel grid to tensor
tensor = np.zeros(np.array(voxel_grid.get_voxels()).shape[:3], dtype=np.float32)
for voxel in voxels:
    voxel_indices = voxel_grid.get_voxel_point_indices(voxel.grid_index)
    if len(voxel_indices) == 0:
        voxel.color = np.array([0, 0, 0], dtype=np.float32)
    else:
        voxel_points = points[voxel_indices]
        voxel.color = np.mean(voxel_points[:, 3:6], axis=0).astype(np.float32)
    tensor[voxel.grid_index] = voxel.color

# Reshape tensor to match expected input shape of CNN
tensor = np.expand_dims(tensor, axis=0)
tensor = np.expand_dims(tensor, axis=1)
print(tensor)
print(tensor.shape)

In [9]:
len(plydata['vertex'].data['y'])

54010

In [1]:
import pickle

def load_preprocessed_data(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

from process_data import save_image_pkl, save_lidar_pkl, load_fish_lidar
        

In [2]:
saved_folder_path = 'C:\\\\Users\\\\szxkd\\\\Desktop\\\\mmml\\\\camelmera\\\\models\\\\gym\\\\data'
save_image_pkl(saved_folder_path)

Processing: C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\T1
7
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [3]:
save_lidar_pkl(saved_folder_path)

Processing: C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\T1
7
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [4]:
image_tensor = load_preprocessed_data("C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\\T1\\image_lcam_fish.pkl")
lidar_tensor = load_preprocessed_data("C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\\T1\\lidar_rcam_fish.pkl")

In [6]:
print(image_tensor[0].shape)
print(lidar_tensor[0].shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import os
import pickle
from torchvision import transforms
from tqdm import tqdm
import timm
from sklearn.decomposition import PCA

# If you have more than 1 trajectory (x trajectories), you need to run this script x times, each time increase the trajectory_index by 1
trajectory_index = 0

class IMU_Image_Depth_Dataset(Dataset):
    def __init__(self, goal_position, load_from_file=False,preprocessed_data_path=None, data_dir=None):
        if load_from_file:
            if preprocessed_data_path != None and os.path.exists(preprocessed_data_path):
                print('Loading preprocessed data from file...')
                with open(preprocessed_data_path, 'rb') as f:
                    self.datasets = pickle.load(f)
            else:
                load_from_file = False
        if not load_from_file:
            if os.path.exists(data_dir):
                print('Preprocessing data...')
                self.datasets = self.load_data(data_dir, goal_position)
                print('Saving preprocessed data to file...')
                if preprocessed_data_path == None:
                    preprocessed_data_path = os.path.join(data_dir, f"preprocessed_data_v{trajectory_index}.pkl")
                self.save_preprocessed_data(self.datasets, preprocessed_data_path)
            else:
                print('Can not get preprocessed data...')

    def load_data(self, data_dir, goal_position):
        all_datasets = []

        for folder in os.listdir(data_dir)[trajectory_index:trajectory_index+1]:
            trajectory_folder_path = os.path.join(data_dir, folder)
            if not os.path.isdir(trajectory_folder_path):
                continue
            states = []
            actions = []
            rewards = []
            positions = []
            print(f'Processing folder: {trajectory_folder_path}')

            # check if contains imu.npy image_lcam_fish.pkl depth_lcam_fish.pkl pose_lcam_front.pkl
            imu_npy_path = os.path.join(trajectory_folder_path, "imu.npy")
            image_pkl_path = os.path.join(trajectory_folder_path, "image_lcam_fish.pkl")
            depth_pkl_path = os.path.join(trajectory_folder_path, "depth_lcam_fish.pkl")
            pose_file_path = os.path.join(trajectory_folder_path, 'pose_lcam_front.txt')

            if not os.path.exists(imu_npy_path):
                print(f"ERROR: generate {trajectory_folder_path}/imu.npy first")
                continue
            if not os.path.exists(image_pkl_path):
                print(f"ERROR: generate {trajectory_folder_path}/image_lcam_fish.pkl first")
                continue
            if not os.path.exists(depth_pkl_path):
                print(f"ERROR: generate {trajectory_folder_path}/depth_lcam_fish.pkl first")
                continue
            if not os.path.exists(pose_file_path):
                print(f"ERROR: download {trajectory_folder_path}/pose_lcam_front.txt first")
                continue
            
            print(f'loading {imu_npy_path}...')
            imu_data_list = np.load(imu_npy_path)

            print(f'loading {image_pkl_path}...')
            with open(image_pkl_path, 'rb') as file:
                image_list = pickle.load(file)
            print(f'loading {depth_pkl_path}...')
            with open(depth_pkl_path, 'rb') as file:
                depth_list = pickle.load(file)
            print(f'loading {pose_file_path}...')
            with open(pose_file_path) as f:
                for line in f.readlines():
                    values = line.strip().split()
                    x, y, z = map(float, values[:3])
                    positions.append(np.array([x, y, z]))
            positions = np.array(positions)  # Convert positions to a numpy array

            assert len(image_list) == len(depth_list)
            assert len(image_list)-1 == imu_data_list.shape[0]
            assert len(image_list) == positions.shape[0]

            model_name = 'vit_base_patch16_224'
            vit_model = timm.create_model(model_name, pretrained=True)
            vit_model.eval()

            image_pca = PCA(n_components=36)
            depth_pca = PCA(n_components=36)

            for idx in tqdm(range(len(image_list)-1)):
                img_batch = image_list[idx]
                depth_batch = depth_list[idx]

                with torch.no_grad():
                    img_embedding = vit_model.forward_features(img_batch) # last_hidden_state of the ViT model
                    img_embedding = img_embedding.reshape(197, 768)
                    image_pca.fit(img_embedding)
                    img_embedding = image_pca.transform(img_embedding) # (197, 36)
                    img_embedding = img_embedding.reshape(-1) # shape: (7092,)

                    depth_embedding = vit_model.forward_features(depth_batch) # last_hidden_state of the ViT model
                    depth_embedding = depth_embedding.reshape(197, 768)
                    depth_pca.fit(depth_embedding)
                    depth_embedding = depth_pca.transform(depth_embedding) #
                    depth_embedding = depth_embedding.reshape(-1) # shape: (7092,)
                state = np.hstack((img_embedding,depth_embedding, imu_data_list[idx], positions[idx]))  # Stack the embeddings and positions horizontally
                states.append(state)

                if idx > 0:
                    action = positions[idx] - positions[idx - 1]
                    actions.append(action)

                    reward = -np.linalg.norm(positions[idx] - goal_position) # Labeling Reward as negative distance to goal
                    rewards.append(reward)
            all_datasets.append({
                      'observations': np.array(states),
                      'actions': np.array(actions),
                      'rewards': np.array(rewards)
                      })
            print("all_datasets len", len(all_datasets))
        return all_datasets


    def save_preprocessed_data(self, datasets, preprocessed_data_path):
        with open(preprocessed_data_path, 'wb') as f:
          pickle.dump(datasets, f)
          print(f'Saved preprocessed data {len(datasets)} to {preprocessed_data_path}')

    def __len__(self):
        # Return the total number of images
        return len(self.datasets)

    def __getitem__(self, idx):
        return self.datasets[idx]

# Example usage of the custom dataset and dataloader
# Define the image directory and labels
data_dir = '/content/drive/MyDrive/tartanairv2filtered'
goal_position = np.array([10, 10, 10])  

# Create an instance of the custom dataset
dataset = IMU_Image_Depth_Dataset(goal_position=goal_position, load_from_file=False, preprocessed_data_path=None, data_dir=data_dir)

# Create a dataloader for batching and shuffling
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

# Iterate over the dataloader during training
for batch_idx, data in enumerate(dataloader):
    # Perform training operations on the batch of data
    print(batch_idx)
    print(len(data))
    print(data['observations'].shape)
    print(data['actions'].shape)
    print(data['rewards'].shape)
    pass